In [1]:
# define the model
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Activation, Add, Concatenate, Lambda, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import backend as K
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [2]:
conv_1 = Conv2D(filters=32, kernel_size=(11, 11), activation="relu")
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
conv_2 = Conv2D(filters=32, kernel_size=(5, 5), activation="relu")
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
conv_3 = Conv2D(filters=64, kernel_size=(3, 3), activation="relu")
cnn_do_1 = Dropout(0.8)
conv_4 = Conv2D(filters=64, kernel_size=(3, 3), activation="relu")
cnn_do_2 = Dropout(0.8)
pool_3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
flatten = Flatten()
dense = Dense(1024, activation='relu')

def CNNnet(x):
    x = conv_1(x)
    x = pool_1(x)
    x = conv_2(x)
    x = pool_2(x)
    x = conv_3(x)
    x = cnn_do_1(x)
    x = conv_4(x)
    x = cnn_do_2(x)
    x = pool_3(x)
    x = flatten(x)
    x = dense(x)
    return x

preprocess_layer = Lambda(lambda x: preprocess_input(x), output_shape=(224,224,3))
mobilenet = MobileNet(input_shape=(224, 224, 3), weights='imagenet', include_top=False, pooling='avg')

h_1_1 = Dense(256, activation='relu')
h_1_2 = Dense(256, activation='relu')
bn_1_1 = BatchNormalization()
bn_1_2 = BatchNormalization()
do_1_1 = Dropout(0.5)
do_1_2 = Dropout(0.5)
s = Dense(1)
def aesthetic_net(img):
    cnn_output = CNNnet(img)
    mobilenet_output = mobilenet(preprocess_layer(img))
    
    x1 = h_1_1(cnn_output)
    x1 = bn_1_1(x1)
    x1 = do_1_1(x1)
    
    x2 = h_1_2(mobilenet_output)
    x2 = bn_1_2(x2)
    x2 = do_1_2(x2)
    
    merge_feature = Concatenate()([x1, x2])
    
    return s(merge_feature)
    

img_a = Input(shape=(224, 224, 3))
img_b = Input(shape=(224, 224, 3))

s_a = aesthetic_net(img_a)
s_b = aesthetic_net(img_b)

ns_b = Lambda(lambda x: -x, output_shape=(1,))(s_b)
diff = Add()([s_a, ns_b])

output = Activation("sigmoid")(diff)

ranknet = Model(inputs=[img_a, img_b], outputs=output)

for layer in mobilenet.layers:
    layer.trainable = False

#optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ranknet.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

ranknet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 214, 214, 32)  11648       input_2[0][0]                    
                                                                   input_3[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 107, 107, 32)  0           conv2d_1[0][0]          

In [3]:
# define the data
from keras.preprocessing.image import *
import os

img_path = "../data/img/"
def load_img_array(imgid, target_size):
    filename = img_path+"%s.jpg"%imgid
    img = image.load_img(filename, target_size=target_size)
    x = image.img_to_array(img)
    return x

# 自定义DirectoryIterator类，可以返回自定义的label
class CustomDirectoryIterator(DirectoryIterator):  
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x1 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_x2 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, ) , dtype=K.floatx())
        # build batch of image data
        for i, j in enumerate(index_array):
            fname =  self.filenames[j]
            fname = os.path.basename(fname)
            fname, _ = os.path.splitext(fname)
            imgA, imgB, cmpret = fname.split("_")
            x1 = load_img_array(imgA, self.target_size)
            x2 = load_img_array(imgB, self.target_size)
            batch_x1[i] = x1
            batch_x2[i] = x2
            batch_y[i] = int(cmpret)
        return [batch_x1, batch_x2], batch_y

# 定义批处理的数据集大小：较小的batch_size可以增加权重调整的次数，同时节省内存的开销
batch_size = 32 

# 图片预处理工具类
train_IDG = ImageDataGenerator(vertical_flip=True, zoom_range=0.1)
valid_IDG = ImageDataGenerator()

# 从目录文件中流式读取数据，避免训练中一次性加载爆内存
train_batch = CustomDirectoryIterator("./data/train/", train_IDG, 
                                      target_size=(224, 224), batch_size=batch_size, shuffle=True)
valid_batch = CustomDirectoryIterator("./data/valid/", valid_IDG, 
                                      target_size=(224, 224), batch_size=batch_size, shuffle=True)

Found 375764 images belonging to 1 classes.
Found 4277 images belonging to 1 classes.


In [ ]:
for _epoch in range(10):
    ranknet.fit_generator(train_batch, steps_per_epoch=train_batch.samples // batch_size, epochs=1,
                       validation_data=valid_batch, validation_steps=valid_batch.samples // batch_size)
    ranknet.save_weights("./model/ranknet_ft_epoch%02d.h5"%_epoch)

Epoch 1/1
  180/11742 [..............................] - ETA: 5622s - loss: 0.7776 - acc: 0.5908

In [ ]:
ranknet.save_weights("./data/ranknet_ft/ranknet_ft.h5")